In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score
from keras.utils import to_categorical
from sklearn.preprocessing import OrdinalEncoder

%matplotlib inline
import matplotlib.pyplot as plt


In [2]:
data_exp=pd.read_csv('BikeSharing_Bluebikes2022.csv')
nat_hol = ['2022-01-01','2022-01-17','2022-02-21','2022-04-17','2022-04-18','2022-05-08','2022-05-30','2022-06-19','2022-06-20','2022-07-04']

data_exp['is_Holiday'] = [ 1 if data_exp.iloc[i]['starttime'][0:10] in nat_hol else 0 for i in range(len(data_exp))]
data_w = pd.read_csv("Boston 2022-01-01 to 2022-08-31.csv")
drop = ['dew', 'sunrise','sunset','moonphase','conditions', 'description','stations','sealevelpressure',
'windgust','severerisk', 'uvindex', 'solarenergy', 'solarradiation','preciptype','winddir','name']
data_w = data_w.drop(columns = drop)
data_w['datetime'] = pd.to_datetime(data_w['datetime'])

In [3]:
from datetime import datetime as dt

data_exp['starttime'] = pd.to_datetime(data_exp['starttime'], format='%Y-%m-%d %H:%M:%S')
data_exp['datetime'] = pd.to_datetime(data_exp['starttime']).dt.date
data_exp['datetime'] = pd.to_datetime(data_exp['datetime'])

data_merge = data_exp.merge(data_w, how='left', on='datetime')

In [4]:
data_merge.columns
data_merge = data_merge.drop(columns=(['Unnamed: 0']))

In [5]:
data_merge.columns


Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'postal code', 'is_Holiday', 'datetime', 'tempmax', 'tempmin', 'temp',
       'feelslikemax', 'feelslikemin', 'feelslike', 'humidity', 'precip',
       'precipprob', 'precipcover', 'snow', 'snowdepth', 'windspeed',
       'cloudcover', 'visibility', 'icon'],
      dtype='object')

In [6]:
data_merge['start station id'].unique()

array([178, 189,  94,  19, 107,  36,  58,  60,   4, 377, 342, 491, 362,
        43, 121, 381,  10,  81, 332, 328, 233, 176,  55,  80,  12,  68,
        32, 202,  54,  74,  39,  66, 356, 116, 398,  42, 352, 330, 426,
        67, 225, 234,  91,  76, 161, 386, 374, 345, 437, 487,  40, 415,
       149, 183,  23,  71,   6,  72, 131, 118,  87,  86,  25,  97, 119,
       338,  69, 359, 446, 239,  22,  47,  26, 125,  33, 157, 105, 471,
        41, 174,  30,  16, 400, 177, 191, 152, 378, 218, 179, 403,  75,
       443, 379,  17,  46,   7,  11, 226, 363, 479, 190, 536, 141,  78,
       181, 139, 380, 417,  95,  90, 364, 549, 529,  44, 151, 376, 100,
        24, 137, 104, 389,  20,  96, 108,  98, 272, 187,  21, 541, 370,
        59, 393, 163, 413,  65, 215, 175, 414, 186, 318,  93, 344, 554,
       472,  63, 169, 196,  51, 419, 331,  84, 478, 515, 115, 213, 372,
       361, 192,  31, 136, 197, 138, 110, 273,  37,  49, 228, 456,  73,
       206,  70, 407, 194, 193, 440,   9, 480, 102, 327,   8,   

In [7]:
list = []
for i in data_merge['start station id'].unique():
    count = data_merge[data_merge['start station id']==i].count()[0]
    list.append(count)

maxim = data_merge['start station id'].unique()[list.index(max(list))]
print(maxim)
data_merge[data_merge['start station id']==maxim].count()[0]



67


49127

In [8]:
data_merge.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'postal code', 'is_Holiday', 'datetime', 'tempmax', 'tempmin', 'temp',
       'feelslikemax', 'feelslikemin', 'feelslike', 'humidity', 'precip',
       'precipprob', 'precipcover', 'snow', 'snowdepth', 'windspeed',
       'cloudcover', 'visibility', 'icon'],
      dtype='object')

In [9]:
data_merge = data_merge[data_merge['start station id']==maxim]
data_nn = data_merge.drop(columns=['start station id',
       'start station name', 'start station latitude',
       'start station longitude','end station name',
       'end station latitude', 'end station longitude','bikeid','postal code', 'stoptime'])

In [10]:
ran = data_nn['starttime'].iloc[3]
ran.hour
(data_nn['starttime'].iloc[0].hour >= 0) & (data_nn['starttime'].iloc[0].hour < 6)

True

In [11]:
range(len(data_nn))
# data_nn['tripduration'].iloc[0]=2
# data_nn['tripduration'].iloc[0]

range(0, 49127)

In [12]:
data_nn['time_of_day'] = 0
for i in range(len(data_nn)):
    if (data_nn['starttime'].iloc[i].hour >= 6) & (data_nn['starttime'].iloc[i].hour < 12):
        data_nn['time_of_day'].iloc[i] = 'morning'
    elif (data_nn['starttime'].iloc[i].hour >= 12) & (data_nn['starttime'].iloc[i].hour < 18):
        data_nn['time_of_day'].iloc[i] = 'afternoon'
    elif (data_nn['starttime'].iloc[i].hour >= 18) & (data_nn['starttime'].iloc[i].hour < 24):
        data_nn['time_of_day'].iloc[i] = 'evening'
    elif (data_nn['starttime'].iloc[i].hour >= 0) & (data_nn['starttime'].iloc[i].hour < 6):
        data_nn['time_of_day'].iloc[i] = 'night'


C:\Users\mirun\AppData\Local\Temp\ipykernel_2180\3777789447.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_nn['time_of_day'].iloc[i] = 'night'


In [13]:
data_nn = data_nn.drop(columns=['starttime','datetime'])

In [14]:
data_nn.columns

Index(['tripduration', 'end station id', 'usertype', 'is_Holiday', 'tempmax',
       'tempmin', 'temp', 'feelslikemax', 'feelslikemin', 'feelslike',
       'humidity', 'precip', 'precipprob', 'precipcover', 'snow', 'snowdepth',
       'windspeed', 'cloudcover', 'visibility', 'icon', 'time_of_day'],
      dtype='object')

In [15]:
data_nn_dummies = pd.get_dummies(data_nn, columns=['time_of_day', 'icon', 'usertype'])

In [16]:
data_nn_dummies_std = data_nn_dummies.copy()

In [17]:
col_std = ['tripduration', 'tempmax',
       'tempmin', 'temp', 'feelslikemax', 'feelslikemin', 'feelslike',
       'humidity', 'precip', 'precipprob', 'precipcover', 'snow', 'snowdepth',
       'windspeed', 'cloudcover', 'visibility']

data_nn_dummies_std[col_std]= (data_nn_dummies_std[col_std] - data_nn_dummies_std[col_std].mean()) / data_nn_dummies_std[col_std].std()

In [22]:
data_nn_dummies_std

,tripduration,end station id,is_Holiday,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,humidity,...,time_of_day_evening,time_of_day_morning,time_of_day_night,icon_clear-day,icon_cloudy,icon_partly-cloudy-day,icon_rain,icon_snow,usertype_Customer,usertype_Subscriber
62,0.070575,67,1,-0.919488,-0.462015,-0.741829,-0.780282,-0.359703,-0.587352,2.489714,...,0,0,1,0,0,0,1,0,0,1
75,-0.036324,139,1,-0.919488,-0.462015,-0.741829,-0.780282,-0.359703,-0.587352,2.489714,...,0,0,1,0,0,0,1,0,0,1
76,-0.036764,139,1,-0.919488,-0.462015,-0.741829,-0.780282,-0.359703,-0.587352,2.489714,...,0,0,1,0,0,0,1,0,0,1
83,-0.030284,55,1,-0.919488,-0.462015,-0.741829,-0.780282,-0.359703,-0.587352,2.489714,...,0,0,1,0,0,0,1,0,0,1
136,0.022945,36,1,-0.919488,-0.462015,-0.741829,-0.780282,-0.359703,-0.587352,2.489714,...,0,0,1,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2305598,0.016716,96,0,0.952752,1.172287,1.029337,0.900253,1.093995,0.959238,0.045236,...,1,0,0,0,0,0,1,0,0,1
2305608,-0.002851,143,0,0.952752,1.172287,1.029337,0.900253,1.093995,0.959238,0.045236,...,1,0,0,0,0,0,1,0,0,1
2305623,-0.003984,143,0,0.952752,1.172287,1.029337,0.900253,1.093995,0.959238,0.045236,...,1,0,0,0,0,0,1,0,0,1
2305661,-0.046643,53,0,0.952752,1.172287,1.029337,0.900253,1.093995,0.959238,0.045236,...,1,0,0,0,0,0,1,0,0,1


In [56]:
uniques = (data_nn_dummies_std['end station id'].unique()).tolist()
uniques.sort()
data_nn_dummies_std['categorical_target'] = [uniques.index(data_nn_dummies_std['end station id'].iloc[i]) for i in range(len(data_nn_dummies_std))]


In [62]:
data_nn_dummies_std = data_nn_dummies_std.drop(columns=['end station id'])

In [63]:
target = data_nn_dummies_std['categorical_target']
data_nn_dummies_std.set_index('categorical_target')
target.unique()

array([ 59, 125,  48,  31, 291,  98,  60, 150,  53,  67,  97, 232, 180,
        39, 105,  87,  81, 178, 236, 230,   9, 323, 309, 243,  68, 211,
        66, 216, 156,  38,  64,  95, 239, 223, 161,  88,  72, 157, 127,
       151,  85,  29,  75, 152,  79,   7, 258,  89, 149,  80, 208,  18,
        22, 209,  35, 261, 292,  47,  28, 240, 270, 324, 184,  36, 286,
       294,   4,   8, 162, 203, 234, 198,  14,  19, 226,  90,  33, 250,
       254, 106, 140, 165, 148, 265,  65, 100, 237, 249,   1, 129, 253,
        94, 181,  16,  10, 276,  77,  44, 185,  84, 205, 251,  58, 238,
       175,  73, 182, 327,  78, 206, 235, 248, 174, 119, 303,  32, 268,
       107, 326,  62, 213,  70, 186,  12, 167, 124,  37, 130,  42, 138,
       154, 204,  15,  69, 302,  34,  86, 120, 207, 101, 222,  92, 179,
        52,  51, 281,  61,  49,  13, 192, 153,  83, 147, 329, 301,  25,
       159,  40, 289,   2, 218, 221, 196, 212,  99, 314, 231,   6, 133,
        63, 319, 134,  24, 242, 278, 194, 287,  26, 228, 141, 26

In [64]:
split = int(len(data_nn_dummies_std)*0.6)
split_val = int(len(data_nn_dummies_std)*0.7)
x_train = data_nn_dummies_std[:split]
y_train = target[:split]
y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
x_val = data_nn_dummies_std[split: split_val]
y_val = target[split: split_val]
y_val = np.asarray(y_val).astype('float32').reshape((-1,1))

x_test = data_nn_dummies_std[split_val:]
y_test = target[split_val:]
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))


C:\Users\mirun\AppData\Local\Temp\ipykernel_2180\2643619492.py:4: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_train = target[:split]
C:\Users\mirun\AppData\Local\Temp\ipykernel_2180\2643619492.py:7: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_val = target[split: split_val]
C:\Users\mirun\AppData\Local\Temp\ipykernel_2180\2643619492.py:11: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *l

In [65]:
print(len(x_train), len(x_val) ,len(x_test),len(x_train)+len(x_val)+len(x_test) == len(data_nn_dummies_std))

29476 4912 14739 True


In [66]:
num_features = x_train.shape[1]
num_output = len(target.unique())
num_output


337

In [72]:
x_train.shape[1]

29

In [99]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

# define the keras model
model = Sequential()
model.add(Dense(800, input_shape = (x_train.shape[1],), activation='relu'))
# model.add(Dropout(rate=0.5))
# model.add(Dense(600, activation='relu'))
# model.add(Dropout(rate=0.5))
model.add(Dense(500, activation='relu'))
# model.add(Dropout(rate=0.5))
model.add(Dense(num_output, activation='softmax'))

In [109]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the keras model on the dataset
model.fit(x_train, y_train, epochs=50, batch_size=500, validation_data=(x_val,y_val))



Epoch 1/50
59/59 [==============================] - 1s 14ms/step - loss: 1.7017 - accuracy: 0.6298 - val_loss: 1.2448 - val_accuracy: 0.7140
Epoch 2/50
59/59 [==============================] - 1s 13ms/step - loss: 0.4992 - accuracy: 0.8284 - val_loss: 1.2290 - val_accuracy: 0.7392
Epoch 3/50
59/59 [==============================] - 1s 13ms/step - loss: 0.4744 - accuracy: 0.8327 - val_loss: 1.2147 - val_accuracy: 0.7362
Epoch 4/50
59/59 [==============================] - 1s 13ms/step - loss: 0.4783 - accuracy: 0.8311 - val_loss: 1.2484 - val_accuracy: 0.7307
Epoch 5/50
59/59 [==============================] - 1s 12ms/step - loss: 0.4802 - accuracy: 0.8288 - val_loss: 1.2539 - val_accuracy: 0.7296
Epoch 6/50
59/59 [==============================] - 1s 13ms/step - loss: 0.4807 - accuracy: 0.8293 - val_loss: 1.2620 - val_accuracy: 0.7250
Epoch 7/50
59/59 [==============================] - 1s 13ms/step - loss: 0.4740 - accuracy: 0.8339 - val_loss: 1.2903 - val_accuracy: 0.7323
Epoch 8/50
59

In [110]:
prediction = model.predict(x_test)

461/461 [==============================] - 1s 2ms/step


In [111]:
loss, accuracy =  model.evaluate(x_test, y_test, verbose=2)

print("Accuracy   : {:5.2f} ".format(accuracy))

461/461 - 1s - loss: 2.5743 - accuracy: 0.7222 - 981ms/epoch - 2ms/step
Accuracy   :  0.72 
